# BioExplorer - Astrocytes
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [55]:
from bioexplorer import BioExplorer, AstrocyteDisplacementParams, Vector2

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load astrocytes

In [56]:
assembly_name = 'Astrocytes'
be.remove_assembly(assembly_name)
vasculature_assembly = be.add_assembly(assembly_name)
vasculature_model = be.add_astrocytes(
    assembly_name=assembly_name,
    population_name='Astrocytes',
    vasculature_population_name='Vasculature',
    population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_BEZIER,
    load_micro_domain=False,
    load_dendrites=True,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    realism_level=be.MORPHOLOGY_REALISM_LEVEL_ALL,
    sql_filter='guid=7178',
    displacement_params=AstrocyteDisplacementParams(
        soma=Vector2(0.1, 0.5), section=Vector2(0.5, 5.0),
        end_foot=Vector2(0.3, 1.0)
    )
)

In [57]:
status = be.reset_camera()

In [58]:
status = core.set_renderer(
    background_color=[0, 0, 0],
    current='advanced',subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_distance = 100.0
params.shadows = 1.0
params.soft_shadows = 1.0
params.epsilon_factor = 1.0
params.max_bounces = 1
params.show_background = False
params.exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()

In [59]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids'][:-1]
    palette = sns.color_palette(palette_name, be.NB_MATERIALS_PER_MORPHOLOGY)

    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        color = palette[mid]
        if mid == be.ASTROCYTE_MATERIAL_MITOCHONDRION:
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.25)
        elif mid == be.ASTROCYTE_MATERIAL_NUCLEUS:
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        elif mid == be.ASTROCYTE_MATERIAL_SOMA:
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        elif mid == be.ASTROCYTE_MATERIAL_END_FOOT:
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        elif mid == be.ASTROCYTE_MATERIAL_MICRO_DOMAIN:
            opacities.append(0.5)
            shading_modes.append(be.SHADING_MODE_ELECTRON)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        else:
            # Membrane
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_CARTOON)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
            
        colors.append(color)
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [60]:
import seaborn as sns
model_ids = be.get_model_ids()['ids']
nb_models = len(model_ids)
for model_id in model_ids:
    set_morphology_materials(model_id, 'Set3')
status = core.set_renderer()